# Calculate the area of each cell of a grid

In [1]:
import nes
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

## 1. Create dataset

### Define grid details

This corresponds to a regular grid for CALIOPE IP

In [3]:
projection='lcc'
lat_1 = 37
lat_2 = 43
lon_0 = -3
lat_0 = 40
nx = 10
ny = 15
inc_x = 4000
inc_y = 4000
x_0 = -807847.688
y_0 = -797137.125
nessy = nes.create_nes(comm=None, info=False, projection=projection,
                       lat_1=lat_1, lat_2=lat_2, lon_0=lon_0, lat_0=lat_0, 
                       nx=nx, ny=ny, inc_x=inc_x, inc_y=inc_y, x_0=x_0, y_0=y_0)

We add data to our recently created grid and coordinate bounds, so we can calculate the cell area using CDO

In [4]:
nessy.variables['aux'] = {'data': np.ones((1, 1, len(nessy.y['data']), len(nessy.x['data'])))}

In [5]:
nessy.create_shapefile()

,geometry
FID,
0,"POLYGON ((-11.58393 32.47507, -11.54169 32.478..."
1,"POLYGON ((-11.54169 32.47851, -11.49944 32.481..."
2,"POLYGON ((-11.49944 32.48192, -11.45719 32.485..."
3,"POLYGON ((-11.45719 32.48533, -11.41494 32.488..."
4,"POLYGON ((-11.41494 32.48871, -11.37268 32.492..."
...,...
145,"POLYGON ((-11.42882 32.99135, -11.38628 32.994..."
146,"POLYGON ((-11.38628 32.99473, -11.34373 32.998..."
147,"POLYGON ((-11.34373 32.99809, -11.30118 33.001..."


We write the NetCDF file

In [6]:
nessy.to_netcdf('lcc_grid.nc')

## 2. Calculate cell area with NES

We calculate the area of each cell according to Huilier's theorem, as in CDO (written in C)

Reference: https://earth.bsc.es/gitlab/ces/cdo/-/blob/master/src/grid_area.c

In [7]:
nessy.calculate_grid_area()
nessy.cell_measures['cell_area']

{'data': array([[15830419.37602062, 15830657.31464759, 15830893.9818806 ,
         15831129.37902967, 15831363.50736617, 15831596.3681775 ,
         15831827.96270714, 15832058.29223234, 15832287.3580128 ,
         15832515.16128502],
        [15832888.43944878, 15833125.4618577 , 15833361.21722112,
         15833595.70683101, 15833828.93195019, 15834060.89386689,
         15834291.59385652, 15834521.03319032, 15834749.2131148 ,
         15834976.13488405],
        [15835346.79239709, 15835582.8966068 , 15835817.73809013,
         15836051.31816116, 15836283.63810146, 15836514.69919126,
         15836744.50269644, 15836973.04988706, 15837200.34204102,
         15837426.38041017],
        [15837794.42326939, 15838029.60727211, 15838263.53291627,
         15838496.20148725, 15838727.61427387, 15838957.77256098,
         15839186.67763537, 15839414.3307679 , 15839640.73320756,
         15839865.88623092],
        [15840231.32048458, 15840465.58230139, 15840698.59010649,
         15840930.

In [8]:
nessy.to_netcdf('lcc_grid_nes_area.nc')

## 4. Calculate cell area with CDO

In the terminal:

```
module load CDO

cdo gridarea lcc_grid.nc lcc_grid_cdo_area.nc
```

## 6. Compare results from NES and CDO

### Merge results

We create the `results` dataframe to store the cell areas calculated to compare NES

In [9]:
results = nessy.shapefile
results 

,geometry
FID,
0,"POLYGON ((-11.58393 32.47507, -11.54169 32.478..."
1,"POLYGON ((-11.54169 32.47851, -11.49944 32.481..."
2,"POLYGON ((-11.49944 32.48192, -11.45719 32.485..."
3,"POLYGON ((-11.45719 32.48533, -11.41494 32.488..."
4,"POLYGON ((-11.41494 32.48871, -11.37268 32.492..."
...,...
145,"POLYGON ((-11.42882 32.99135, -11.38628 32.994..."
146,"POLYGON ((-11.38628 32.99473, -11.34373 32.998..."
147,"POLYGON ((-11.34373 32.99809, -11.30118 33.001..."


Since the file resulting from CDO (`lcc_grid_cdo_area.nc`) is not CF compliant, we get its values using xarray

In [10]:
results['area_cdo'] = xr.open_dataset('/gpfs/projects/bsc32/models/NES_tutorial_data/lcc_grid_cdo_area.nc').cell_area.values.ravel()

In [11]:
results['area_nes'] = nessy.cell_measures['cell_area']['data'].ravel()

In [12]:
results

,geometry,area_cdo,area_nes
FID,,,
0,"POLYGON ((-11.58393 32.47507, -11.54169 32.478...",1.584877e+07,1.583042e+07
1,"POLYGON ((-11.54169 32.47851, -11.49944 32.481...",1.584900e+07,1.583066e+07
2,"POLYGON ((-11.49944 32.48192, -11.45719 32.485...",1.584924e+07,1.583089e+07
3,"POLYGON ((-11.45719 32.48533, -11.41494 32.488...",1.584948e+07,1.583113e+07
4,"POLYGON ((-11.41494 32.48871, -11.37268 32.492...",1.584971e+07,1.583136e+07
...,...,...,...
145,"POLYGON ((-11.42882 32.99135, -11.38628 32.994...",1.588347e+07,1.586512e+07
146,"POLYGON ((-11.38628 32.99473, -11.34373 32.998...",1.588369e+07,1.586534e+07
147,"POLYGON ((-11.34373 32.99809, -11.30118 33.001...",1.588390e+07,1.586556e+07


### Differences

Differences are relatively big because we consider the radius minor and major axes of the Earth to be:
```
Minor = 6356752.3142 
Major = 6378137
```
These come from the definition of WGS84 in the Cartopy source code and give us a relative difference of approximately 0.12%. 

If we change them to be as it is defined in CDO:
```
6371000.0
```
The relative difference is about 0.03%.

#### Absolute differences

In [13]:
(results['area_nes']-results['area_cdo'])

FID
0     -18346.478591
1     -18346.515863
2     -18346.552797
3     -18346.589390
4     -18346.625647
           ...     
145   -18346.387763
146   -18346.402289
147   -18346.416586
148   -18346.430659
149   -18346.444509
Length: 150, dtype: float64

In [14]:
((results['area_nes']-results['area_cdo'])).mean()

np.float64(-18346.737697358058)

In [15]:
((results['area_nes']-results['area_cdo'])).max()

np.float64(-18346.311654422432)

In [16]:
((results['area_nes']-results['area_cdo'])).min()

np.float64(-18347.010924736038)

#### Relative differences

In [17]:
(results['area_nes']-results['area_cdo'])*100/results['area_cdo']

FID
0     -0.115760
1     -0.115758
2     -0.115757
3     -0.115755
4     -0.115754
         ...   
145   -0.115506
146   -0.115505
147   -0.115503
148   -0.115502
149   -0.115500
Length: 150, dtype: float64

In [18]:
((results['area_nes']-results['area_cdo'])*100/results['area_cdo']).mean()

np.float64(-0.11563013960524898)

In [19]:
((results['area_nes']-results['area_cdo'])*100/results['area_cdo']).max()

np.float64(-0.11550023822922381)

In [20]:
((results['area_nes']-results['area_cdo'])*100/results['area_cdo']).min()

np.float64(-0.11575966708957625)